## Automate Business Metric Reporting with DataCamp Workspace

You can consult the solution by visiting `notebook-solution.ipynb` in the file browser.


In [19]:
# Install necessary packages
!pip install slack_sdk
from slack_sdk import WebClient
import os
from datetime import datetime
print (datetime.now())


Defaulting to user installation because normal site-packages is not writeable
2025-08-27 18:56:51.425657


In [7]:
SELECT @@version

,?column?
0,Microsoft SQL Server 2019 (RTM-CU21) (KB502580...


## Total sales by month

In [8]:
-- Get total sales over time
SELECT SUM(quantity*list_price*(1-discount)) AS total_sales
FROM sales.orders o
INNER JOIN sales.order_items oi ON o.order_id=oi.order_id

,total_sales
0,7.689117e+06


In [40]:
SELECT 
	DATEFROMPARTS ( DATEPART(year, order_date), DATEPART(month, order_date), 1) AS month,
	SUM(quantity * list_price * (1 - discount)) AS total_sales
FROM sales.orders o
INNER JOIN sales.order_items oi ON o.order_id = oi.order_id
WHERE FORMAT(o.order_date,'yyy-MM') ='2018-04'
GROUP BY DATEFROMPARTS ( DATEPART(year, order_date), DATEPART(month, order_date), 1)

,month,total_sales
0,2018-04-01 00:00:00+00:00,817921.8604


Use plotply instead Visualize bar chart for sales_over_time, showing Month on x axis, total_sales on y axis is , dark blue color

In [18]:
import plotly.express as px
from IPython.display import display

# Create a bar chart using Plotly
fig = px.bar(sales_over_time, x='month', y='total_sales', 
             title='Total Sales Over Time',
             labels={'month': 'Month', 'total_sales': 'Total Sales'},
             color_discrete_sequence=['darkblue'])

# Update layout for better visualization
fig.update_layout(xaxis_title='Month', yaxis_title='Total Sales',
                  xaxis_tickformat='%b %Y',
                  bargap=0.2)

# Display the figure
fig.show()

In [10]:
SELECT SUM(quantity * list_price * (1 - discount)) AS total_sales,
    FORMAT(order_date, 'yyyy-MM') AS month
FROM sales.orders o
INNER JOIN sales.order_items oi ON o.order_id = oi.order_id
GROUP BY FORMAT(order_date, 'yyyy-MM')

,total_sales,month
0,215146.4241,2016-01-01
1,156112.3228,2016-02-01
2,180600.3285,2016-03-01
3,167144.0512,2016-04-01
4,205270.0091,2016-05-01
5,210562.1245,2016-06-01
6,199556.8089,2016-07-01
7,225657.3767,2016-08-01
8,273091.6097,2016-09-01
9,212078.0805,2016-10-01


## Surface top sellers

In [34]:
-- Get top three of best sellers in April 2018 (by staff)
SELECT TOP 3
	CONCAT(s.first_name,' ', s.last_name) AS seller,
	SUM(oi.quantity * oi.list_price * (1 - oi.discount)) AS total_sales
FROM sales.orders o
INNER JOIN sales.order_items oi ON o.order_id = oi.order_id
INNER JOIN sales.staffs s ON o.staff_id = s.staff_id
WHERE FORMAT(o.order_date,'yyy-MM') ='2018-04'
GROUP BY CONCAT(s.first_name,' ', s.last_name)
ORDER BY total_sales DESC


,seller,total_sales
0,Venita Daniel,275594.3638
1,Genna Serrano,174986.5221
2,Marcelene Boyer,161670.5474


In [1]:
-- Get top three of best sellers in April 2018 (by product)
SELECT 
p.product_id,
p.product_name, 
FORMAT(o.order_date,'yyyy-MM') AS month,
SUM(oi.quantity * oi.list_price * (1 - oi.discount)) AS total_sales
FROM production.products p
INNER JOIN sales.order_items oi ON p.product_id = oi.product_id
INNER JOIN sales.orders o ON o.order_id = oi.order_id
WHERE FORMAT(o.order_date,'yyyy-MM') = '2018-04'
GROUP BY p.product_id, p.product_name, FORMAT(o.order_date,'yyyy-MM')
ORDER BY total_sales DESC
OFFSET 0 ROWS FETCH NEXT 3 ROWS ONLY;

,product_id,product_name,month,total_sales
0,155,Trek Domane SLR 9 Disc - 2018,2018-04-01,32759.9727
1,207,Trek Boone 7 Disc - 2018,2018-04-01,21039.9474
2,49,Trek Domane SL 6 - 2017,2018-04-01,21034.9399


## Report metrics to Slack

In [5]:
!pip install slack_sdk

Defaulting to user installation because normal site-packages is not writeable


In [4]:
import os

slack_oauth_token = os.environ["SLACK_OAUTH_TOKEN"]

,month,total_sales
17,2018-12-01 00:00:00+00:00,6516.9667


In [43]:
from slack_sdk import WebClient
import os

from babel.numbers import format_currency
last_sales_number = sales_over_time[sales_over_time['month'] == sales_over_time['month'].max()]['total_sales'].values[0]

sellers_list = top_sellers.apply(lambda x: f". {x['seller']} ({format_currency(x['total_sales'],'USD')})", axis="columns")
sellers_overview = "\n".join(sellers_list)

text = f"""The last month we did {format_currency(last_sales_number, 'USD')} in sales.

Our sales all stars:
{sellers_overview}

_<https://www.datacamp.com/datalab/w/a2e8895f-1345-403f-8e80-5710f9f55d8a/edit|Link to Digital Collab Hub>_
"""

client = WebClient(token=os.environ["SLACK_OAUTH_TOKEN"])
client.chat_postMessage(channel="sales-tracking", text='Hi Sales Team!  ' + text)